In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("../../data/df_rfm.csv")

# Display the first few rows of the data
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.40
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.80
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.60
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
713597,581587,22613,PACK_OF_20_SPACEBOY_NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
713598,581587,22899,CHILDREN'S_APRON_DOLLY_GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
713599,581587,23254,CHILDRENS_CUTLERY_DOLLY_GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
713600,581587,23255,CHILDRENS_CUTLERY_CIRCUS_PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [18]:
# Compter le nombre de pays uniques pour chaque Customer_ID
unique_countries = df.groupby('Customer_ID')['Country'].nunique()

# Afficher les Customer_ID qui ont plus d'un pays
unique_countries[unique_countries > 1]

Customer_ID
12370    2
12394    2
12413    2
12417    2
12422    2
12423    2
12429    2
12431    2
12449    2
12455    2
12652    2
Name: Country, dtype: int64

In [19]:
unique_countries.value_counts()

1    5801
2      11
Name: Country, dtype: int64

In [20]:
df[df['Customer_ID'] == 12423]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
208001,520737,22179,SET_10_LIGHTS_NIGHT_OWL,4,2010-08-31 09:31:00,6.75,12423,Denmark,27.00
208002,520737,22468,BABUSHKA_LIGHTS_STRING_OF_10,4,2010-08-31 09:31:00,6.75,12423,Denmark,27.00
208003,520737,22138,BAKING_SET_9_PIECE_RETROSPOT,6,2010-08-31 09:31:00,4.95,12423,Denmark,29.70
208004,520737,22617,BAKING_SET_SPACEBOY_DESIGN,3,2010-08-31 09:31:00,4.95,12423,Denmark,14.85
208005,520737,22557,PLASTERS_IN_TIN_VINTAGE_PAISLEY,12,2010-08-31 09:31:00,1.65,12423,Denmark,19.80
...,...,...,...,...,...,...,...,...,...
713202,581493,20724,RED_RETROSPOT_CHARLOTTE_BAG,10,2011-12-09 10:10:00,0.85,12423,Belgium,8.50
713203,581493,23204,CHARLOTTE_BAG_APPLES_DESIGN,10,2011-12-09 10:10:00,0.85,12423,Belgium,8.50
713204,581493,21108,FAIRY_CAKE_FLANNEL_ASSORTED_COLOUR,18,2011-12-09 10:10:00,0.79,12423,Belgium,14.22
713205,581493,22252,BIRDCAGE_DECORATION_TEALIGHT_HOLDER,12,2011-12-09 10:10:00,1.25,12423,Belgium,15.00


In [21]:
from datetime import timedelta

# Convertir InvoiceDate de l'objet au format datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Obtenir la date maximale
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)

# Agréger les données (RFM : Récence, Fréquence, Valeur monétaire)
rfm = df.groupby(['Customer_ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Calcul de la récence
    'Invoice': 'count',  # Comptage du nombre de transactions (fréquence)
    'TotalSales': 'sum'  # Somme des ventes (valeur monétaire)
})

# Renommer les colonnes
rfm.rename(columns={'InvoiceDate': 'Recency',
                   'Invoice': 'Frequency',
                   'TotalSales': 'MonetaryValue'}, inplace=True)

# Afficher les premières lignes des données
rfm



,Recency,Frequency,MonetaryValue
Customer_ID,,,
12346,326,21,77327.62
12347,2,216,4620.32
12348,75,46,1658.40
12349,19,161,3319.06
12350,310,13,258.00
...,...,...,...
18283,4,925,2527.45
18284,432,25,391.28
18285,661,10,361.10


In [22]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5812.000000,5812.000000,5812.000000
mean,199.701136,122.780798,2641.060447
std,208.191682,311.484945,13167.396763
min,1.000000,1.000000,0.000000
25%,25.000000,19.000000,307.775000
50%,94.000000,48.000000,791.675000
75%,379.000000,127.250000,2058.422500
max,738.000000,11123.000000,553408.560000


In [23]:
rfm['Country'] = df.groupby('Customer_ID')['Country'].agg(lambda x:x.value_counts().index[0])

rfm

,Recency,Frequency,MonetaryValue,Country
Customer_ID,,,,
12346,326,21,77327.62,United_Kingdom
12347,2,216,4620.32,Iceland
12348,75,46,1658.40,Finland
12349,19,161,3319.06,Italy
12350,310,13,258.00,Norway
...,...,...,...,...
18283,4,925,2527.45,United_Kingdom
18284,432,25,391.28,United_Kingdom
18285,661,10,361.10,United_Kingdom


In [24]:
# cherche l'index de la ligne avec le Customer_ID 12346
rfm.loc[12423]

Recency                1
Frequency            134
MonetaryValue    2021.85
Country          Belgium
Name: 12423, dtype: object

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer un sous-ensemble de données avec les 3 variables
rfm_rfm = rfm[['Recency', 'Frequency', 'MonetaryValue']]

# Créer une figure avec trois sous-graphiques
fig = make_subplots(rows=3, cols=1, subplot_titles=('Recency', 'Frequency', 'MonetaryValue'))

# Ajouter les graphiques de boîtes à chaque sous-graphique
fig.add_trace(go.Box(y=rfm_rfm['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(y=rfm_rfm['Frequency'], name='Frequency'), row=2, col=1)
fig.add_trace(go.Box(y=rfm_rfm['MonetaryValue'], name='MonetaryValue'), row=3, col=1)

# Mettre à jour le titre et les dimensions de la figure
fig.update_layout(title='Distribution des variables RFM',
                  height=800, width=600)

# Afficher la figure interactive dans le notebook
fig.show()


In [26]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5812.000000,5812.000000,5812.000000
mean,199.701136,122.780798,2641.060447
std,208.191682,311.484945,13167.396763
min,1.000000,1.000000,0.000000
25%,25.000000,19.000000,307.775000
50%,94.000000,48.000000,791.675000
75%,379.000000,127.250000,2058.422500
max,738.000000,11123.000000,553408.560000


In [27]:
# Define the upper limit for each variable
recency_upper_limit = rfm['Recency'].quantile(0.95)
frequency_upper_limit = rfm['Frequency'].quantile(0.95)
monetary_value_upper_limit = rfm['MonetaryValue'].quantile(0.95)

# Remove the outliers
rfm = rfm[(rfm['Recency'] <= recency_upper_limit) & 
          (rfm['Frequency'] <= frequency_upper_limit) & 
          (rfm['MonetaryValue'] <= monetary_value_upper_limit)]

In [28]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5095.000000,5095.000000,5095.000000
mean,185.967223,79.955054,1296.775072
std,182.652094,88.332807,1475.476388
min,1.000000,1.000000,0.000000
25%,29.000000,19.000000,313.565000
50%,96.000000,46.000000,751.690000
75%,365.000000,109.000000,1671.855000
max,622.000000,447.000000,8438.340000


In [29]:
# Créer un sous-ensemble de données avec les 3 variables
rfm_rfm = rfm[['Recency', 'Frequency', 'MonetaryValue']]

# Créer une figure avec trois sous-graphiques
fig = make_subplots(rows=3, cols=1, subplot_titles=('Recency', 'Frequency', 'MonetaryValue'))

# Ajouter les graphiques de boîtes à chaque sous-graphique
fig.add_trace(go.Box(y=rfm_rfm['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(y=rfm_rfm['Frequency'], name='Frequency'), row=2, col=1)
fig.add_trace(go.Box(y=rfm_rfm['MonetaryValue'], name='MonetaryValue'), row=3, col=1)

# Mettre à jour le titre et les dimensions de la figure
fig.update_layout(title='Distribution des variables RFM',
                  height=800, width=600)

# Afficher la figure interactive dans le notebook
fig.show()

In [30]:
rfm.shape

(5095, 4)

In [31]:
# met le rfm dans un fichier csv
rfm.to_csv("../../data/df_rfm_model.csv" , index = 'Customer_ID')